In [1]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [19]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

### calculating the win/loss ratio's correlation with the population of the city it is in for the NHL using 2018 data.

In [37]:
nhl_df = pd.read_csv("assets/nhl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]

In [38]:
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


In [39]:
cities.head()

,Metropolitan area,Country,Pop. rank,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL
0,New York City,United States,1,20153634,9,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3],11,Red Bulls New York City FC,—
1,Los Angeles,United States,2,13310447,8,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks,10,Galaxy Los Angeles FC[note 5],—
2,San Francisco Bay Area,United States,6,6657982,6,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7],7,Earthquakes,—
3,Chicago,United States,3,9512999,5,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks,6,Fire,—
4,Dallas–Fort Worth,United States,4,7233323,4,Cowboys,Rangers,Mavericks,Stars,5,FC Dallas,—


In [40]:
cities = cities.iloc[:-1,[0,3,5,6,7,8]]

In [41]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3]
1,Los Angeles,13310447,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [44]:
def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]',string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]',string1).group(), "")

In [77]:
def get_area(team):
    for each in list(nhl_cities.index.values):
        if team in each:
            return nhl_cities.at[each, "Metropolitan area"]

In [46]:
cities["NHL"] = cities["NHL"].apply(lambda x:clear_data(x))

In [48]:
cities["NHL"].head()

0    Rangers Islanders Devils
1                 Kings Ducks
2                      Sharks
3                  Blackhawks
4                       Stars
Name: NHL, dtype: object

In [54]:
nhl_cities = cities[["Metropolitan area","NHL"]].set_index("NHL")
nhl_cities.head()

,Metropolitan area
NHL,
Rangers Islanders Devils,New York City
Kings Ducks,Los Angeles
Sharks,San Francisco Bay Area
Blackhawks,Chicago
Stars,Dallas–Fort Worth


In [55]:
nhl_cities = nhl_cities.drop(["—",""],axis=0)
nhl_cities

,Metropolitan area
NHL,
Rangers Islanders Devils,New York City
Kings Ducks,Los Angeles
Sharks,San Francisco Bay Area
Blackhawks,Chicago
Stars,Dallas–Fort Worth
Capitals,"Washington, D.C."
Flyers,Philadelphia
Bruins,Boston
Wild,Minneapolis–Saint Paul


In [60]:
print(nhl_df.shape)
nhl_df.head()

(171, 15)


,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


In [61]:
nhl_df = nhl_df[nhl_df["year"]==2018].drop([0,9,18,26],axis=0)

In [63]:
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL


In [66]:
population = cities[["Metropolitan area","Population (2016 est.)[8]"]].set_index("Metropolitan area")

In [67]:
population.head()

,Population (2016 est.)[8]
Metropolitan area,
New York City,20153634
Los Angeles,13310447
San Francisco Bay Area,6657982
Chicago,9512999
Dallas–Fort Worth,7233323


In [68]:
nhl_df["team"] = nhl_df["team"].apply(lambda x:x[:-1].strip() if x.endswith("*") else x.strip())

In [72]:
nhl_df["team"].head()

1    Tampa Bay Lightning
2          Boston Bruins
3    Toronto Maple Leafs
4       Florida Panthers
5      Detroit Red Wings
Name: team, dtype: object

In [73]:
nhl_df["area"] = nhl_df["team"].apply(lambda x:x.split(" ")[-1])

In [76]:
nhl_df["area"].head()

1    Lightning
2       Bruins
3        Leafs
4     Panthers
5        Wings
Name: area, dtype: object

In [78]:
nhl_df["area"] = nhl_df["area"].apply(lambda x:get_area(x))

In [80]:
nhl_df["area"]

1             Tampa Bay Area
2                     Boston
3                    Toronto
4      Miami–Fort Lauderdale
5                    Detroit
6                   Montreal
7                     Ottawa
8                    Buffalo
10          Washington, D.C.
11                Pittsburgh
12              Philadelphia
13                  Columbus
14             New York City
15                   Raleigh
16             New York City
17             New York City
19                 Nashville
20                  Winnipeg
21    Minneapolis–Saint Paul
22                    Denver
23                 St. Louis
24         Dallas–Fort Worth
25                   Chicago
27                 Las Vegas
28               Los Angeles
29    San Francisco Bay Area
30               Los Angeles
31                   Calgary
32                  Edmonton
33                 Vancouver
34                   Phoenix
Name: area, dtype: object

In [81]:
out=[]

In [84]:
for group,frame in nhl_df.groupby("area"):
    total_wins = np.sum(pd.to_numeric(frame["W"]))
    total_losses = np.sum(pd.to_numeric(frame["L"]))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_losses
    out_dict = {
        "Area":group,
        "Ratio":ratio
    }
    out.append(out_dict)

In [85]:
new_df = pd.DataFrame(out)

In [87]:
new_df.head()

,Area,Ratio
0,Boston,2.500000
1,Buffalo,0.555556
2,Calgary,1.057143
3,Chicago,0.846154
4,Columbus,1.500000


In [89]:
new_df = new_df.set_index("Area")

In [90]:
out_df = pd.merge(new_df , population , how="inner" , left_index=True , right_index=True)

In [94]:
out_df["Population (2016 est.)[8]"] = pd.to_numeric(out_df["Population (2016 est.)[8]"])

In [95]:
out_df.head()

,Ratio,Population (2016 est.)[8]
Boston,2.500000,4794447
Buffalo,0.555556,1132804
Calgary,1.057143,1392609
Chicago,0.846154,9512999
Columbus,1.500000,2041520


In [96]:
def nhl_correlation():
    population_by_region = []
    win_loss_by_region = []
    population_by_region = out_df["Population (2016 est.)[8]"].to_list()
    win_loss_by_region = out_df["Ratio"].to_list()
    
    assert len(population_by_region) == len(win_loss_by_region),"Q1: Your List Must be the Same Length"
    assert len(population_by_region) == 28,"There should be 28 Teams being Analysed  for the NHL"
    
    return np.float64(stats.pearsonr(population_by_region,win_loss_by_region)[0])
    

In [98]:
def get_nhl_data():
    return out_df

In [99]:
print(nhl_correlation())
print(get_nhl_data())

-0.07333781676467474
                           Ratio  Population (2016 est.)[8]
Boston                  2.500000                    4794447
Buffalo                 0.555556                    1132804
Calgary                 1.057143                    1392609
Chicago                 0.846154                    9512999
Columbus                1.500000                    2041520
Dallas–Fort Worth       1.312500                    7233323
Denver                  1.433333                    2853077
Detroit                 0.769231                    4297617
Edmonton                0.900000                    1321426
Las Vegas               2.125000                    2155664
Los Angeles             1.648148                   13310447
Miami–Fort Lauderdale   1.466667                    6066387
Minneapolis–Saint Paul  1.730769                    3551036
Montreal                0.725000                    4098927
Nashville               2.944444                    1865298
New York City      